# Skill estimation using Stan

Let's see how we can use a simple **continuous** variable model to predict a hidden "skill level" of players based on their performance in a collection of games against each other.  We need data (the games and outcomes), and a model of how skill translates into these outcomes (e.g., higher skilled players have a better chance of winning).

In [10]:
import numpy as np
import pystan
import matplotlib.pyplot as plt
%matplotlib inline

We'll use "stan" to define the model; this will compile an inference engine which can then perform Monte Carlo (and some other) approximate inference procedures to give estimates.

We'll describe the (prior) distribution of skill levels as Gaussian, and then model the probability of Player A winning over Player B as a logistic function of the two players' skill differences, with a scaling coefficient (which we can set or try to learn):

In [11]:
skill_model = """
data {
  int<lower=1> N;             # Total number of players
  int<lower=1> E;             # number of games
  real<lower=0> scale;        # scale value for probability computation
  int<lower=0,upper=1> win[E]; # PA wins vs PB
  int PA[E];                  # player info between each game
  int PB[E];                  # 
}
parameters {
  vector [N] skill;           # skill values for each player
}

model{
  for (i in 1:N){ skill[i]~normal(0,3); }
  for (i in 1:E){
    win[i] ~ bernoulli_logit( (scale)*(skill[PA[i]]-skill[PB[i]]) );
  }   # win probability is a logit function of skill difference
}
"""

Now, compile the model.  We'll save a version of it so we don't have to recompile every time.

In [12]:
import pickle
try:     # load it if already compiled
    sm = pickle.load(open('skill_model.pkl', 'rb'))
except:  # ow, compile and save compiled model
    sm = pystan.StanModel(model_code = skill_model)
    with open('skill_model.pkl', 'wb') as f: pickle.dump(sm, f)

We also need the observed data: number of players and games, which pairs played each game, and who won:

In [13]:
skill_data = {
    'N': 3,
    'E': 4,
    'scale': 0.3,
    'win':[1, 1, 1, 0],
    'PA': [1, 1, 2, 1],
    'PB': [3, 3, 3, 2]
}
# Player 1 & 3 played & P1 won; then again; then P2 & P3 (P2 wins), etc.

Now, we can perform MCMC on the model, and extract the samples:

In [14]:
fit = sm.sampling(data=skill_data, iter=10000, chains=4)

In [15]:
samples = fit.extract()

If we just want the mean estimate for each player's skill level, just take the empirical average over the samples:

In [16]:
samples['skill'].mean(0)

array([ 0.57593548,  1.832701  , -2.45314344])

If we want to predict which player will win, we might use a direct estimator of that quantity based on the sample values:

In [17]:
# Player 0 vs Player 1 prediction:
def logit(z): return 1./(1.+np.exp(-z))

# Use our model's win probability function (logistic of scaled difference)
#  using the predicted skill difference for each sample:
prob = logit( skill_data['scale']*(samples['skill'][:,0]-samples['skill'][:,1]) ).mean()

print(prob)

0.42315631660497527
